In [1]:
import psycopg2 as pg2
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from Player_rank import Player_ranker
from sqlalchemy import create_engine

/anaconda3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [ ]:
'''
Here we're loading in all of the traditional Fantasy basketball stats into SQL, using window function to get a ranking by minutes

'''

In [ ]:
'''IF you have open connections run the following in the psql command prompt:

SELECT pg_terminate_backend(pid) FROM pg_stat_activity WHERE datname = 'nba_capstone';

'''

In [2]:
conn = pg2.connect(dbname = 'postgres',host='localhost')


In [3]:
conn.autocommit = True

In [4]:
cur = conn.cursor()


In [5]:
'''Note for some reason you can not access the database if it is not all lowercase'''

cur.execute('DROP DATABASE IF EXISTS nba_capstone;')  # Makes sure there is not already a class_example database and removes is if there is
cur.execute('CREATE DATABASE nba_capstone;')



In [6]:
cur.close()
conn.close()

In [7]:
conn = pg2.connect(dbname = 'nba_capstone',host='localhost')
conn.autocommit = True

In [8]:
cur = conn.cursor()


In [9]:
query = '''
        CREATE TABLE NBA_stats (
            Season integer, 
            Player varchar(50), 
            Pos varchar(10),
            Age int,
            Tm varchar(15),
            G int,
            GS int,
            MP float,
            FG float,
            FGA float,
            FG_Percentage float,
            Threes_Made float,
            Threes_Attempted float,
            Three_Percentage float,
            Twos_Made float,
            Twos_Attempted float,
            Twos_Percentage float,
            eff_FG_Percentage float,
            FTM float,
            FTA float,
            FT_Percentage float,
            ORB float,
            DRB float,
            Rebounds float,
            AST float,
            STL float,
            BLK float,
            TOV float,
            Fouls float,
            Points float
        );
        '''

cur.execute(query)

In [10]:
# import os
# current_directory_path = os.getcwd()
# current_directory_path

query = '''
        COPY NBA_stats 
        FROM '/Users/rcheer/Desktop/Galvanize/Capstone/Fantasy-Basketball-Capstone-Project/NBA stats.csv' 
        DELIMITER ',' 
        CSV HEADER;
        '''



cur.execute(query)

In [11]:
query = '''
        CREATE TABLE nba_advanced (
            Season integer, 
            Player varchar(50), 
            Pos varchar(10),
            Age int,
            Tm varchar(15),
            G int,
            total_MP float,
            PER float,
            True_Shooting float,
            Three_Attempt_Rate float,
            FT_rate float,
            ORB_Percentage float,
            DRB_Percentage float,
            Rebound_Percentage float,
            Assist_Percentage float,
            Steal_Percentage float,
            Block_Percentage float,
            Turnover_Percentage float,
            Usage_Percentage float,
            Offensive_WinShares float,
            Defensive_WinShares float,
            WinShares float,
            WinShares_Per48 float,
            Offensive_BoxPlusMinus float,
            Defensive_BoxPlusMinus float,
            BoxPlusMinus float,
            Value_overReplacement float
        );
        '''

cur.execute(query)

In [12]:
query = '''
        COPY nba_advanced 
        FROM '/Users/rcheer/Desktop/Galvanize/Capstone/Fantasy-Basketball-Capstone-Project/NBA Advanced.csv' 
        DELIMITER ',' 
        CSV HEADER;
        '''



cur.execute(query)

In [13]:
'''
save for later
d.points,d.rebounds,d.ast,d.stl,d.blk,d.tov,d.fg_percentage,d.FT_percentage
'''


query1 = '''
            DROP TABLE IF EXISTS players;
            CREATE TABLE players AS
            select season,player,max(G) as Games from NBA_stats where Tm!='TOT' group by season,player;
            
            DROP TABLE IF EXISTS y_predictions;
            CREATE TABLE y_predictions AS
            select d.season,d.player,d.pos,d.age,MAX(case when p.player is not null then d.Tm else NULL end) as StartingTeam,SUM(G) as Games,SUM(GS) as GS,
            max(MP) as minutes
            from NBA_stats d
            left join players p
                on d.season = p.season
                and d.player = p.player
                and d.G = p.Games
            where d.Tm!='TOT'
            group by d.season,d.player,d.pos,d.age;
            
            DROP TABLE IF EXISTS rank_by_minutes;
            CREATE TEMPORARY TABLE rank_by_minutes AS
            select y.*,n.points,n.rebounds,n.ast,n.stl,n.blk,n.tov,n.threes_made,n.fg,n.fga,n.ftm,n.fta,
            row_number() over(partition by n.season order by MP*G desc) as min_rank from NBA_stats n
            inner join y_predictions y
                ON n.player = y.player
                and n.season = y.season
                and n.Tm=y.startingTeam;
            
            select * from rank_by_minutes        
                
        '''
cur.execute(query1)
data = cur.fetchall()
df = pd.DataFrame(np.array(data))
df.columns = ['season','player','position','age','team','gamesPlayed','gamesStarted','minutes'
              ,'points','rebounds','assists','steals','blocks','turnovers','threes_made','FGM','FGA','FTM','FTA','min_rank']


In [14]:
df['min_rank']=pd.to_numeric(df['min_rank'])
df['points']=pd.to_numeric(df['points'])
df['rebounds']=pd.to_numeric(df['rebounds'])
df['assists']=pd.to_numeric(df['assists'])
df['steals']=pd.to_numeric(df['steals'])
df['blocks']=pd.to_numeric(df['blocks'])
df['turnovers']=pd.to_numeric(df['turnovers'])
df['threes_made']=pd.to_numeric(df['threes_made'])
df['FGM']=pd.to_numeric(df['FGM'])
df['FGA']=pd.to_numeric(df['FGA'])
df['FTM']=pd.to_numeric(df['FTM'])
df['FTA']=pd.to_numeric(df['FTA'])
df['gamesPlayed']=pd.to_numeric(df['gamesPlayed'])
df['gamesStarted']=pd.to_numeric(df['gamesStarted'])
df['minutes']=pd.to_numeric(df['minutes'])
df['age']=pd.to_numeric(df['age'])
df['season']=pd.to_numeric(df['season'])


In [72]:
#top200_min = df.copy()[df.copy()['min_rank']<=200]

In [75]:
#top200_min.groupby('season').mean().reset_index()[['season','points','rebounds','assists','steals','blocks','turnovers','threes_made']]
#top200_min.groupby('season').std()

,season,points,rebounds,assists,steals,blocks,turnovers,threes_made
0,2008,13.3005,5.1235,2.7870,0.9150,0.5690,1.7035,0.8780
1,2009,12.9850,5.1080,2.8490,0.9120,0.5600,1.6965,0.8265
2,2010,12.9225,5.0185,2.8135,0.9085,0.5685,1.6790,0.8380
3,2011,12.1830,4.8915,2.6840,0.9225,0.5725,1.6805,0.8030
4,2012,12.5550,5.0520,2.8840,0.9585,0.6125,1.7410,0.8800
5,2013,13.0510,5.1265,2.8310,0.9505,0.5420,1.7375,0.9715
6,2014,12.3805,4.9890,2.7145,0.9230,0.5390,1.6315,0.9600
7,2015,12.8715,5.1935,2.7720,0.9530,0.5900,1.6785,1.0530
8,2016,13.3550,5.0420,2.8950,0.9230,0.5340,1.6195,1.2170
9,2017,13.2915,5.0950,2.9355,0.9240,0.5585,1.6825,1.3200


In [15]:
test = Player_ranker(df)
test.get_category_dist()
test.cat_mean

In [18]:
test.assign_values()

In [19]:
value_copy = test.value.copy()

In [20]:
value_copy[value_copy['player']=='Victor Oladipo']

,season,player,value_tot,value_points,value_rebounds,value_assists,value_blocks,value_steals,value_turnovers,value_threes,value_fg,value_ft
2334,2013,Victor Oladipo,-1.005537,0.148978,-0.390006,0.629596,-0.082715,1.555845,-1.956027,-0.090524,-0.924816,0.104132
2785,2014,Victor Oladipo,2.033454,1.151138,-0.311694,0.742659,-0.458483,1.826969,-1.594947,0.307911,-0.495046,0.864947
3308,2015,Victor Oladipo,3.000245,0.599503,-0.153985,0.554812,0.386922,1.547666,-0.543205,0.407329,-0.477088,0.678292
3802,2016,Victor Oladipo,-0.092168,0.431124,-0.287012,-0.140983,-0.503694,0.721964,-0.220695,0.815165,-0.566801,-0.341237
4245,2017,Victor Oladipo,6.742591,1.807176,0.040703,0.694323,0.518411,3.597157,-1.511767,0.905503,0.389094,0.301991


In [21]:
#copy value data back into SQL
engine = create_engine("postgresql://@localhost/nba_capstone")

value_copy.to_sql(name='value', con=engine, if_exists = 'replace', index=False)




In [58]:
value_copy.head()

,season,player,Total_value,value_points,value_rebounds,value_assists,value_blocks,value_steals,value_turnovers,value_threes,value_fg,value_ft,season_rank
0,2008,Andre Iguodala,1.569149,1.022902,0.233549,1.209232,-0.313073,1.720216,-1.392654,0.164932,0.201353,-1.277308,391.0
1,2008,Joe Johnson,2.248938,1.506499,-0.293101,1.449827,-0.683573,0.464584,-1.113145,1.381647,-1.036752,0.572951,403.0
2,2008,O.J. Mayo,0.491912,0.967102,-0.536170,0.198732,-0.683573,0.464584,-1.532409,1.246456,-0.633993,1.001183,364.0
3,2008,Antawn Jamison,3.459954,1.655298,1.529918,-0.426816,-0.498323,0.715710,0.284401,0.705694,0.083254,-0.589183,423.0
4,2008,Raymond Felton,-0.135656,0.167306,-0.536170,1.882899,-0.313073,1.469090,-1.532409,-0.240639,-1.364914,0.332253,338.0


In [33]:
query = '''
        DROP TABLE IF EXISTS player_value;
        CREATE TABLE player_value AS
        select ROW_NUMBER() OVER(PARTITION BY season ORDER BY value_tot DESC),* from value;
        
        DROP TABLE IF EXISTS value;
        
        select * from player_value;
        
        '''


cur.execute(query)
ranking_data = cur.fetchall()
df_2 = pd.DataFrame(np.array(ranking_data))
cols_value = ['playerrank']
for item in (list(value_copy.columns)):
    cols_value.append(item)
cols_value
df_2.columns=cols_value


In [44]:
for i in df_2.columns:
    if i!='player':
        df_2[i]=pd.to_numeric(df_2[i])

In [50]:
df_2[df_2['season']==2017].sort_values(by='playerrank', ascending=True)

,playerrank,season,player,value_tot,value_points,value_rebounds,value_assists,value_blocks,value_steals,value_turnovers,value_threes,value_fg,value_ft
4219,1,2017,Anthony Davis,13.069292,2.728406,2.327843,-0.323373,4.382346,1.403769,-0.642579,-0.719758,2.572502,1.340136
4220,2,2017,Stephen Curry,10.977767,2.415188,0.001938,1.610250,-0.769567,1.647478,-1.635937,3.343394,1.073741,3.291282
4221,3,2017,Kevin Durant,10.555627,2.415188,0.660945,1.254056,2.665042,-0.545910,-1.635937,1.369863,1.809299,2.563082
4222,4,2017,James Harden,10.522310,3.152171,0.118234,2.984140,0.303748,2.134898,-3.374314,2.762944,-0.589500,3.029989
4223,5,2017,Karl-Anthony Towns,9.046572,1.475533,2.793024,-0.272488,1.806390,-0.302200,-0.270069,0.208962,2.213129,1.394291
4224,6,2017,LeBron James,8.050018,2.617858,1.358716,3.136795,0.733074,1.160059,-3.125975,0.557232,2.935558,-1.323300
4225,7,2017,Giannis Antetokounmpo,7.982516,2.507311,1.901427,0.948747,1.806390,1.403769,-1.635937,-0.835849,2.328774,-0.442116
4226,8,2017,Damian Lillard,7.758411,2.507311,-0.230652,1.864674,-0.340241,0.428929,-1.387598,2.066403,-0.920536,3.770119
4227,9,2017,Chris Paul,7.259998,0.978069,0.118234,2.526177,-0.769567,1.891188,-0.642579,1.369863,-0.176916,1.965529
4228,10,2017,Jimmy Butler,7.194467,1.641355,0.079468,0.999632,-0.340241,2.622318,-0.145900,-0.139308,0.324360,2.152782


In [48]:
query = '''
        CREATE TABLE points(
        player varchar(50),
        season int,
        Starting_team varchar(25))
        
        '''


cur.execute(query)

0         1
1         2
2         3
3         4
4         5
5         6
6         7
7         8
8         9
9        10
10       11
11       12
12       13
13       14
14       15
15       16
16       17
17       18
18       19
19       20
20       21
21       22
22       23
23       24
24       25
25       26
26       27
27       28
28       29
29       30
       ... 
4729    511
4730    512
4731    513
4732    514
4733    515
4734    516
4735    517
4736    518
4737    519
4738    520
4739    521
4740    522
4741    523
4742    524
4743    525
4744    526
4745    527
4746    528
4747    529
4748    530
4749    531
4750    532
4751    533
4752    534
4753    535
4754    536
4755    537
4756    538
4757    539
4758    540
Name: playerrank, Length: 4759, dtype: int64

In [29]:
cols_value = ['playerrank']

for item in (list(value_copy.columns)):
    cols_value.append(item)
cols_value

['playerrank',
 'season',
 'player',
 'value_tot',
 'value_points',
 'value_rebounds',
 'value_assists',
 'value_blocks',
 'value_steals',
 'value_turnovers',
 'value_threes',
 'value_fg',
 'value_ft']

In [46]:
'''Looking at Top Players'''
value_copy.sort_values(by=['season','Total_value'],ascending=False)

,season,player,Total_value,value_points,value_rebounds,value_assists,value_blocks,value_steals,value_turnovers,value_threes,value_fg,value_ft
4230,2017,Anthony Davis,13.069292,2.728406,2.327843,-0.323373,4.382346,1.403769,-0.642579,-0.719758,2.572502,1.340136
4380,2017,Stephen Curry,10.977767,2.415188,0.001938,1.610250,-0.769567,1.647478,-1.635937,3.343394,1.073741,3.291282
4266,2017,Kevin Durant,10.555627,2.415188,0.660945,1.254056,2.665042,-0.545910,-1.635937,1.369863,1.809299,2.563082
4246,2017,James Harden,10.522310,3.152171,0.118234,2.984140,0.303748,2.134898,-3.374314,2.762944,-0.589500,3.029989
4225,2017,Karl-Anthony Towns,9.046572,1.475533,2.793024,-0.272488,1.806390,-0.302200,-0.270069,0.208962,2.213129,1.394291
4219,2017,LeBron James,8.050018,2.617858,1.358716,3.136795,0.733074,1.160059,-3.125975,0.557232,2.935558,-1.323300
4228,2017,Giannis Antetokounmpo,7.982516,2.507311,1.901427,0.948747,1.806390,1.403769,-1.635937,-0.835849,2.328774,-0.442116
4236,2017,Damian Lillard,7.758411,2.507311,-0.230652,1.864674,-0.340241,0.428929,-1.387598,2.066403,-0.920536,3.770119
4339,2017,Chris Paul,7.259998,0.978069,0.118234,2.526177,-0.769567,1.891188,-0.642579,1.369863,-0.176916,1.965529
4287,2017,Jimmy Butler,7.194467,1.641355,0.079468,0.999632,-0.340241,2.622318,-0.145900,-0.139308,0.324360,2.152782


In [ ]:

            CREATE TEMPORARY TABLE test AS
            select p1.*,sum(p2.GS) as teammates_gamesStarted,Max(case when p1.pos=p2.pos then p2.GS else 0 end) as teammate_same_pos from y_predictions p1
            left join y_predictions p2
                on p1.season = p2.season+1
                and p1.StartingTeam = p2.StartingTeam
            where p1.player!=p2.player 
            group by p1.season,p1.player,p1.pos,p1.age, p1.StartingTeam,p1.Games,p1.GS,p1.minutes;
            
            select t.*,MAX(d.points),MAX(d.rebounds),MAX(d.ast),MAX(d.stl),MAX(d.blk),MAX(d.tov),MAX(d.fg_percentage),MAX(d.FT_percentage),SUM(d.G),SUM(d.GS),MAX(d.MP) as Last_Minutes
            ,max(a.winshares),max(a.winshares_per48),max(a.boxplusminus),max(a.value_overreplacement) from test t
            left join data_2017 d
                ON t.season = d.season+1
                and t.player = d.player
                and d.tm!='TOT'
            left join nba_advanced a
                on a.player = t.player
                and a.season+1 = t.season
                and a.tm!='TOT'
            where (d.MP>20 or (d.MP>10 and d.age<25) or (d.MP is null and d.age<23))
            group by t.season,t.player,t.pos,t.age, t.StartingTeam,t.Games,t.GS,t.minutes,teammates_gamesStarted,teammate_same_pos
            
            select * from data_2017 d
            inner join nba_advanced a
                ON d.player = a.player
            where d.tm!=a.tm

In [210]:
query = '''
        CREATE TABLE final_table(
        season int,
        player varchar(50),
        position varchar(50),
        age int,
        starter int,
        points float,
        rebounds float,
        assists float,
        steals float,
        blocks float,
        Threes float,
        FG float,
        FT float,
        TO float,
        starter_lastyear int,
        Min_lastyear float,
        ChangeMin_lastyear float,
        points_lastyear float,
        pointsPerMin_lastyear float,
        change_pointsPerMin_lastyear float,
        rebounds_lastyear float
        RebPerMin_lastyear float,
        change_RebPerMin_lastyear float,
        assists_lastyear float,
        AstPerMin_lastyear float,
        change_AstPerMin_lastyear float,
        steals_lastyear float,
        StlPerMin_lastyear float,
        change_StlPerMin_lastyear float,
        blocks_lastyear float,
        BlkPerMin_lastyear float,
        change_BlkPerMin_lastyear float,
        FT_LastYear float,
        FT_PreviousYear float,
        FG_LastYear float,
        FG_LastYear float,
        Threes_LastYear float,
        Change_Threes_lastyear float,
        Team_Added_UsageMins float,
        Team_Subtracted_UsageMins float,
        change_usage float,
        Prior_yearPlayerRank float,
        Offensive_WinShares float,
        Defensive_WinShares float,
        Change_Offensive_WinShares float,
        Change_Defensive_WinShares float,
        Offensive_BoxPlusMinus float,
        Change_Offensive_BoxPlusMinus float,
        Defensive_BoxPlusMinus float,
        Change_Defensive_BoxPlusMinus float,
        )
        
        INSERT final_table(season,player,position,age,starter,points,rebounds,assists,steals,blocks,Threes,FG,FT,TO)
        
        CREATE TEMPORARY TABLE Changed_Teams AS
        select d.player,d.pos,d.tm,SUM(a.total_MP*usage_percentage)/(48*82) from data_2017 d
        inner join nba_advanced a
             ON d.player = a.player
             and d.season = a.season+1
        where d.tm!=a.tm
        group by d.player,d.pos,d.tm
        
        
        '''
cur.execute(query)




In [181]:
cur.close()
conn.close

<function connection.close>

In [15]:

df[(df.team=='UTA') &(df.season==2017)].sort_values(by='position')

,season,player,position,age,team,gamesPlayed,gamesStarted,minutes,team_lastSeason_starts,teammate_samepos_starts,...,turnovers,FG_percentage,FT_percentage,lastyear_games,lastyear_GS,last_mins,winshares,winshares_per48,boxplusminus,value_overreplacement
1017,2017,Derrick Favors,C,26,UTA,77,77,28,371,81,...,1.2,0.487,0.615,50,39,23.7,2.7,0.109,-0.1,0.6
1213,2017,Rudy Gobert,C,25,UTA,56,56,32.4,329,1,...,1.8,0.661,0.653,81,81,33.9,14.3,0.25,5.8,5.4
1003,2017,Dante Exum,PG,22,UTA,14,0,16.8,384,49,...,1.2,0.427,0.795,66,26,18.6,1.2,0.046,-2.5,-0.1
1204,2017,Ricky Rubio,PG,27,UTA,77,77,29.3,410,49,...,2.6,0.402,0.891,75,75,32.9,6.1,0.119,0.8,1.7
1079,2017,Jae Crowder,SF,27,UTA,27,0,27.6,410,73,...,1.1,0.463,0.811,72,72,32.4,6.7,0.137,1.6,2.1
1105,2017,Joe Ingles,SF,30,UTA,82,81,31.4,384,73,...,1.3,0.452,0.735,82,26,24,5,0.123,2.7,2.3
1107,2017,Joe Johnson,SF,36,UTA,32,3,21.9,396,73,...,0.9,0.436,0.818,78,14,23.6,3.8,0.099,-0.3,0.8
1237,2017,Thabo Sefolosha,SF,33,UTA,38,6,21.2,410,73,...,0.9,0.441,0.733,62,42,25.7,3.3,0.098,1.2,1.3
1208,2017,Rodney Hood,SG,25,UTA,39,12,27.8,355,0,...,1.1,0.408,0.783,59,55,27,2.6,0.078,-1.2,0.3


In [471]:
df.columns
df['starter'] = ((df['gamesStarted']/df['gamesPlayed'])>0.8).astype(int)

In [485]:
y = df['starter']
X = df[['team_lastSeason_starts','teammate_samepos_starts','boxplusminus','value_overreplacement','age','lastyear_games','lastyear_GS']]
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3)

In [497]:
r2_lr = np.mean(cross_val_score(LogisticRegression(C=1000),X_train,y_train,cv=10,n_jobs=-1))
print('Logistic Regression error: {}'.format(r2_lr))

r2_gb = np.mean(cross_val_score(GradientBoostingClassifier(learning_rate=0.01,n_estimators=250,max_depth = 4,max_features=4),X_train,y_train,cv=5,n_jobs=-1))
print('GradientBoost error: {}'.format(r2_gb))

r2_rf = np.mean(cross_val_score(RandomForestClassifier(n_estimators=500,max_depth = 4,max_features=4),X_train,y_train,cv=5,n_jobs=-1))
print('RandomForest error: {}'.format(r2_rf))

Logistic Regression error: 0.8196708659630006
GradientBoost error: 0.8061484076634157
RandomForest error: 0.8095256131048195


In [500]:
parameters = {'learning_rate':[0.001,0.01,0.05], 'n_estimators':[100,250,500,1000],'max_depth':[4,5,6],'max_features':[4,5,6]}

clf = GridSearchCV(GradientBoostingClassifier(), parameters, cv=5)
clf.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'learning_rate': [0.001, 0.01, 0.05], 'n_estimators': [100, 250, 500, 1000], 'max_depth': [4, 5, 6], 'max_features': [4, 5, 6]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [494]:
clf.best_estimator_

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.01, loss='deviance', max_depth=4,
              max_features=4, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=250,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)

In [495]:
GradientBoostingClassifier?

Init signature: GradientBoostingClassifier(loss='deviance', learning_rate=0.1, n_estimators=100, subsample=1.0, criterion='friedman_mse', min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=3, min_impurity_decrease=0.0, min_impurity_split=None, init=None, random_state=None, max_features=None, verbose=0, max_leaf_nodes=None, warm_start=False, presort='auto')
Docstring:     
Gradient Boosting for classification.

GB builds an additive model in a
forward stage-wise fashion; it allows for the optimization of
arbitrary differentiable loss functions. In each stage ``n_classes_``
regression trees are fit on the negative gradient of the
binomial or multinomial deviance loss function. Binary classification
is a special case where only a single regression tree is induced.

Read more in the :ref:`User Guide <gradient_boosting>`.

Parameters
----------
loss : {'deviance', 'exponential'}, optional (default='deviance')
    loss function to be optimized. 'deviance' refers